In [2]:
import pandas as pd

Из train.csv читаем только первые 1e6 строк, т.к. весь файл не помещается в RAM. Т.к. по заданию необходимо тольлько "проанализировать как можно больше характеристик, влияющих на успеваемость студентов" а не тренировать модель, то этого количества думаю будет достаточно.

In [91]:
path_to_data = '/home/danil/Downloads/data/' # put your data location here
n_rows = 1_000_000

train_df = pd.read_csv(path_to_data + 'train.csv', sep=',', nrows=n_rows)
question_df = pd.read_csv(path_to_data + 'questions.csv', sep=',')
lectures_df = pd.read_csv(path_to_data + 'lectures.csv', sep=',')

In [93]:
len(train_df['row_id'].unique()) == n_rows #убедимся что нет дубликатов

True

Целевой переменной показывающей успеваемость студентов будем считать train_df.answered_correctly. Т.е. если студент правильно отвечает на вопросы, то он успевает

In [94]:
train_df_copy = train_df.copy() # делаем копию чтобы не повредить оригинальные данные

In [95]:
train_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   row_id                          1000000 non-null  int64  
 1   timestamp                       1000000 non-null  int64  
 2   user_id                         1000000 non-null  int64  
 3   content_id                      1000000 non-null  int64  
 4   content_type_id                 1000000 non-null  int64  
 5   task_container_id               1000000 non-null  int64  
 6   user_answer                     1000000 non-null  int64  
 7   answered_correctly              1000000 non-null  int64  
 8   prior_question_elapsed_time     976277 non-null   float64
 9   prior_question_had_explanation  996184 non-null   object 
dtypes: float64(1), int64(8), object(1)
memory usage: 76.3+ MB


Для начала откорректируем типы данных к оригинальным, согласно описанию:

In [96]:
conv = {'user_id': 'int32',
        'content_id': 'int16',
        'content_type_id': 'int8',
        'task_container_id': 'int16',
        'user_answer': 'int8',
        'answered_correctly': 'int8',
        'prior_question_elapsed_time': 'float32',
        'prior_question_had_explanation': 'bool'
        }
train_df_copy = train_df_copy.astype(conv)

Далее, удалим столбец, который дублирует индекс. Это row_id

In [97]:
train_df_copy.drop('row_id', axis=1, inplace=True)

In [98]:
train_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   timestamp                       1000000 non-null  int64  
 1   user_id                         1000000 non-null  int32  
 2   content_id                      1000000 non-null  int16  
 3   content_type_id                 1000000 non-null  int8   
 4   task_container_id               1000000 non-null  int16  
 5   user_answer                     1000000 non-null  int8   
 6   answered_correctly              1000000 non-null  int8   
 7   prior_question_elapsed_time     976277 non-null   float32
 8   prior_question_had_explanation  1000000 non-null  bool   
dtypes: bool(1), float32(1), int16(2), int32(1), int64(1), int8(3)
memory usage: 22.9 MB


В результате такого преобразования получили экономию памяти более чем в 3 раза: 76.3 => 22.9 MB.
! Отметим, что при конвертации столбца prior_question_had_explanation пустые значения заполнились True. (не уверен, что это корректно, но это стандартное поведение функции astype)

Выделим, по описанию, те характиристики, которые прямо могут влиять на успеваемость:
1. timestamp - да (время взаимодействия пользователя с контентом) ;
2. user_id - нет, но это поле может быть использовано для агрегации данных;
3. content_id - нет, это просто ссылка на вопрос или лекцию, но может быть использовано для конструирования признака сложности вопроса. Т.е. если на вопрос часто отвечают правильно, то можно считать его более легким, и наоборот;
4. content_type_id (0-вопрос, 1-лекция) - да, просмотр лекций, очевидно должен влиять на резултаты ответов
5. task_container_id - нет, но может быть использован для создания других полезных признаков. Например, количества вопросов в одном контейнере. Возможно, если контейнер слишком большой, пользователь просто устает, и начинает чаще ошибаться. Похожее поле bundle_id есть в таблице question, но в чем отличие пока не ясно 
6. user_answer - нет, разве что для оценки количества вариантов ответа на отдельный вопрос
7. prior_question_elapsed_time - пока не ясно как можно использовать
8. prior_question_had_explanation - да, это может указывать, что пользователь сомневался в ответе на предыдущий вопрос

# Анализ таблиц по отдельности

Логично разделить данные по лекциям и по вопросам. Эта операция не приведет к потере каких либо данных т.к. строки таблицы train никак не связаны между собой, и невозможно сказать в какой последовательности пользователь изучал лекции и/или отвечал на вопросы.

In [127]:
# Лекции
train_df_copy_lec = train_df_copy[train_df_copy['content_type_id'] == 1].copy()
train_df_copy_lec.describe()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,1.990700e+04,1.990700e+04,19907.000000,19907.0,19907.000000,19907.0,19907.0,0.0
mean,7.897040e+09,1.034608e+07,16677.181243,1.0,712.355704,-1.0,-1.0,NaN
std,1.127003e+10,5.987159e+06,9565.767222,0.0,792.733475,0.0,0.0,NaN
min,2.416820e+05,2.746000e+03,89.000000,1.0,2.000000,-1.0,-1.0,NaN
25%,9.761835e+08,4.980312e+06,8411.000000,1.0,171.000000,-1.0,-1.0,NaN
50%,3.618622e+09,9.793549e+06,16363.000000,1.0,412.000000,-1.0,-1.0,NaN
75%,9.914121e+09,1.556872e+07,24985.000000,1.0,960.000000,-1.0,-1.0,NaN
max,7.680911e+10,2.093825e+07,32736.000000,1.0,5033.000000,-1.0,-1.0,NaN


Наблюдения
- время в милисекундах неудобно для чтения человеком, возмможно, потом лучше сконвертировать в timedelta
- время изучения лекций лежит в пределах от 4-х минут до ~21 часа. В среднем на лекцию пользователь тратит ~2.2 часа

In [100]:
lectures_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lecture_id  418 non-null    int64 
 1   tag         418 non-null    int64 
 2   part        418 non-null    int64 
 3   type_of     418 non-null    object
dtypes: int64(3), object(1)
memory usage: 13.2+ KB


Справочник лекций состоит из 418 записей, и имеет 3 категориальных признака, два из которых выражены как int64, и один object. Пропусков данных нет. Посмотрим, какие категории представлены в признаках

In [101]:
len(lectures_df['tag'].unique()) # количество тегов

151

In [102]:
len(lectures_df['part'].unique()) # количество частей (top level category code for the lecture)

7

In [103]:
lectures_df['type_of'].value_counts(normalize=True)

concept             0.531100
solving question    0.444976
intention           0.016746
starter             0.007177
Name: type_of, dtype: float64

Всего есть четыре типа лекций, на первые два (concept и solving question) приходится более 98% записей справочникаю.

In [136]:
# Вопросы
train_df_copy_ques = train_df_copy[train_df_copy['content_type_id'] == 0].copy()
train_df_copy_ques.describe()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time
count,9.800930e+05,9.800930e+05,980093.000000,980093.0,980093.000000,980093.000000,980093.000000,976277.000000
mean,7.333085e+09,1.016903e+07,5000.238626,0.0,808.009877,1.423019,0.650358,25319.472656
std,1.057230e+10,6.030037e+06,3287.211531,0.0,1029.988408,1.156775,0.476857,19707.429688
min,0.000000e+00,1.150000e+02,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
25%,5.169283e+08,4.700718e+06,1999.000000,0.0,107.000000,0.000000,0.000000,16000.000000
50%,2.805786e+09,9.678259e+06,4996.000000,0.0,390.000000,1.000000,1.000000,21000.000000
75%,1.010340e+10,1.556872e+07,7218.000000,0.0,1115.000000,3.000000,1.000000,29666.000000
max,7.809200e+10,2.094902e+07,13522.000000,0.0,7739.000000,3.000000,1.000000,300000.000000


Наблюдения
- есть как минимум один вопрос с временем ответа 0 милисекунд. Скорее всего это ошибка, и нужно рассматривать такие данные как выбросы
- среднее время на обдумывание ответа составляет 2 часа, что довольно много, и практически совпадает в временем изучения лекций
- максимальное время 21.7 часа, даже больше чем для лекций
- 65% ответов правильные

In [119]:
#проверим, сколько строк с нулевыми значениями в timestsmp
train_df_copy_ques[train_df_copy_ques['timestamp'] == 0].shape[0]

3847

In [105]:
question_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13523 entries, 0 to 13522
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   question_id     13523 non-null  int64 
 1   bundle_id       13523 non-null  int64 
 2   correct_answer  13523 non-null  int64 
 3   part            13523 non-null  int64 
 4   tags            13522 non-null  object
dtypes: int64(4), object(1)
memory usage: 528.4+ KB


В справочнике вопросов 13523 записей, и 4 категориальных признака, 3 из которых выражены числами, один строкой

In [108]:
question_df.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


Заметим, что поле tags (в отличие от справочника лекций) является строкой с числами, разделенными пробелами

In [109]:
question_df['part'].unique() #количество частей (the relevant section of the TOEIC test)

array([1, 2, 3, 4, 5, 6, 7])

In [113]:
len(question_df['bundle_id'].unique())

9765

In [115]:
question_df['bundle_id'].value_counts()

8034     5
7790     5
7260     5
7195     5
7190     5
        ..
5058     1
5057     1
5056     1
5055     1
13522    1
Name: bundle_id, Length: 9765, dtype: int64

Вопросы соединяются в 9765 групп, от 1 до 5 в каждой группе (bundle_id)

# Теперь присоединим справочники к основной таблице

In [140]:
train_df_copy_lec_mer = train_df_copy_lec.merge(lectures_df, how='left', left_on='content_id', right_on='lecture_id')

In [141]:
train_df_copy_lec_mer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19907 entries, 0 to 19906
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   timestamp                       19907 non-null  int64  
 1   user_id                         19907 non-null  int32  
 2   content_id                      19907 non-null  int16  
 3   content_type_id                 19907 non-null  int8   
 4   task_container_id               19907 non-null  int16  
 5   user_answer                     19907 non-null  int8   
 6   answered_correctly              19907 non-null  int8   
 7   prior_question_elapsed_time     0 non-null      float32
 8   prior_question_had_explanation  19907 non-null  bool   
 9   lecture_id                      19907 non-null  int64  
 10  tag                             19907 non-null  int64  
 11  part                            19907 non-null  int64  
 12  type_of                         

Пропусков данных в присоединенных столбцах из lecture нет, значит некорректных ссылок на лекции в train нет

Теперь посмотрим какой типы лекций студенты изучают чаще других

In [134]:
train_df_copy_lec_mer.describe(include='object')

,type_of
count,19907
unique,3
top,concept
freq,14326


это тип concept

Теперь присоединим справочник вопросов question_df к основной таблице train_df_copy_ques

In [137]:
train_df_copy_ques_mer = train_df_copy_ques.merge(question_df, how='left', left_on='content_id', right_on='question_id')

In [142]:
train_df_copy_ques_mer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 980093 entries, 0 to 980092
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   timestamp                       980093 non-null  int64  
 1   user_id                         980093 non-null  int32  
 2   content_id                      980093 non-null  int16  
 3   content_type_id                 980093 non-null  int8   
 4   task_container_id               980093 non-null  int16  
 5   user_answer                     980093 non-null  int8   
 6   answered_correctly              980093 non-null  int8   
 7   prior_question_elapsed_time     976277 non-null  float32
 8   prior_question_had_explanation  980093 non-null  bool   
 9   question_id                     980093 non-null  int64  
 10  bundle_id                       980093 non-null  int64  
 11  correct_answer                  980093 non-null  int64  
 12  part            

In [147]:
# после соединения, можно вывести агрегированные данные по новым столбцам. Например среднее время с гуппировкой по "part"
train_df_copy_ques_mer.groupby('part').mean()['timestamp']

part
1    6.228541e+09
2    7.499765e+09
3    8.591294e+09
4    7.625731e+09
5    6.928887e+09
6    7.938026e+09
7    7.664366e+09
Name: timestamp, dtype: float64

Далее, я бы сконвертировал котегориальные признаки, выраженные числами в тип данных category, и создал несколько новых признаков на основе имеющихся. Но это выходит за рамки задания.